# 几何畸变校正探索

In [2]:
import os,tarfile,sys
import ee
import geemap
import numpy as np
import pandas as pd
import eerepr
import folium
import copy
import math
from tqdm import trange,tqdm
from functools import partial
geemap.set_proxy(port=10809)
# ee.Authenticate()
ee.Initialize(project='ee-mrwurenzhe')
print('geemap version = {}\ngeemap path = {}'.format(geemap.__version__,geemap.__path__))

geemap version = 0.35.3
geemap path = ['c:\\Users\\A\\.conda\\envs\\GEE\\Lib\\site-packages\\geemap']


# 预加载数据

## 预加载SHP数据

In [4]:
Glacial_lake_2015A = ee.FeatureCollection(
    'projects/ee-mrwurenzhe/assets/Glacial_lake/SAR_GLs/2019Gls_SARExt')
    #projects/ee-mrwurenzhe/assets/Glacial_lake/Checkout_polygen

# 计算geometry、质心点、最小包络矩形
Geo_ext = lambda feature: feature.set({
    'Geo': feature.geometry(),
    'Centroid': feature.geometry().centroid(),
    'Rectangle': feature.geometry().bounds()
})

Centrid_set = lambda feature: feature.setGeometry(feature.geometry().centroid())
Rectangle_set = lambda feature: feature.setGeometry(feature.geometry().bounds())
Glacial_lake_2015C = Glacial_lake_2015A.map(Geo_ext).map(Centrid_set)  # 添加属性,修改geometry,计算质心
Glacial_lake_2015R = Glacial_lake_2015A.map(Rectangle_set)       # 计算最小包络矩形

#抽取属性作为list
Glacial_lake_2015A_GeoList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(), ['Geo']).get('list'))
Glacial_lake_2015C_CentriodList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Centroid']).get('list'))
Glacial_lake_2015R_RectangleList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Rectangle']).get('list'))
Num_list = Glacial_lake_2015C_CentriodList.size().getInfo()

In [6]:
Glacial_lake_2015R_RectangleList.get(I)

## 预加载影像数据

In [11]:
START_DATE  = ee.Date('2019-08-01')
END_DATE   = ee.Date('2019-08-30')
DEM = ee.Image('USGS/SRTMGL1_003')
models = ['volume', 'surface', None]
Buffer = 0
Model = models[0]
I = 800

AOI = ee.Feature(Glacial_lake_2015R_RectangleList.get(I)).geometry()
s1_col = (ee.ImageCollection("COPERNICUS/S1_GRD")
          .filter(ee.Filter.eq('instrumentMode', 'IW'))
            .filterBounds(AOI)
            .filterDate(START_DATE, END_DATE))

# Slop correction

In [12]:
#-----------------------------------------------地形校正-----------------------------------------
def slope_correction(collection, elevation, model, buffer=0):
    '''This function applies the slope correction on a collection of Sentinel-1 data

       :param collection: ee.Collection or ee.Image of Sentinel-1
       :param elevation: ee.Image of DEM
       :param model: model to be applied (volume/surface)
       :param buffer: buffer in meters for layover/shadow amsk

       :returns: ee.Image
    '''
    def _volumetric_model_SCF(theta_iRad, alpha_rRad):
        '''Code for calculation of volumetric model SCF
            体积模型
        :param theta_iRad: ee.Image of incidence angle in radians
        :param alpha_rRad: ee.Image of slope steepness in range

        :returns: ee.Image
        '''
        # create a 90 degree image in radians
        ninetyRad = ee.Image.constant(90).multiply(np.pi / 180)

        # model
        nominator = (ninetyRad.subtract(theta_iRad).add(alpha_rRad)).tan()
        denominator = (ninetyRad.subtract(theta_iRad)).tan()
        return nominator.divide(denominator)

    def _surface_model_SCF(theta_iRad, alpha_rRad, alpha_azRad):
        '''Code for calculation of direct model SCF
            表面模型
        :param theta_iRad: ee.Image of incidence angle in radians
        :param alpha_rRad: ee.Image of slope steepness in range
        :param alpha_azRad: ee.Image of slope steepness in azimuth

        :returns: ee.Image
        '''

        # create a 90 degree image in radians
        ninetyRad = ee.Image.constant(90).multiply(np.pi / 180)

        # model
        nominator = (ninetyRad.subtract(theta_iRad)).cos()
        denominator = (alpha_azRad.cos().multiply(
            (ninetyRad.subtract(theta_iRad).add(alpha_rRad)).cos()))

        return nominator.divide(denominator)

    def _erode(image, distance):
        '''Buffer function for raster
            腐蚀算法，输入的图像需要额外的缓冲
        :param image: ee.Image that shoudl be buffered
        :param distance: distance of buffer in meters

        :returns: ee.Image
        '''

        d = (image.Not().unmask(1).fastDistanceTransform(30).sqrt().multiply(
            ee.Image.pixelArea().sqrt()))

        return image.updateMask(d.gt(distance))

    def _masking(alpha_rRad, theta_iRad, buffer):
        '''Masking of layover and shadow
            获取几何畸变区域
        :param alpha_rRad: ee.Image of slope steepness in range
        :param theta_iRad: ee.Image of incidence angle in radians
        :param buffer: buffer in meters

        :returns: ee.Image
        '''
        # layover, where slope > radar viewing angle
        layover = alpha_rRad.lt(theta_iRad).rename('layover')

        # shadow
        ninetyRad = ee.Image.constant(90).multiply(np.pi / 180)
        shadow = alpha_rRad.gt(
            ee.Image.constant(-1).multiply(
                ninetyRad.subtract(theta_iRad))).rename('shadow')

        # add buffer to layover and shadow
        if buffer > 0:
            layover = _erode(layover, buffer)
            shadow = _erode(shadow, buffer)

        # combine layover and shadow
        no_data_mask = layover.And(shadow).rename('no_data_mask')

        return layover.addBands(shadow).addBands(no_data_mask)

    def _correct(image):
        '''
        This function applies the slope correction and adds layover and shadow masks
        '''

        # get the image geometry and projection
        geom = image.geometry()
        proj = image.select(1).projection()

        # 计算雷达方位向，ee.Terrain.aspect()本是用来计算坡度，该函数返回每个像素点相对于正北方向的坡度角度值
        heading = (ee.Terrain.aspect(image.select('angle')).reduceRegion(
            ee.Reducer.mean(), geom, 1000).get('aspect'))

        # Sigma0 to Power of input image，Sigma0是指雷达回波信号的强度,dB转信号强度
        sigma0Pow = ee.Image.constant(10).pow(image.divide(10.0))

        # the numbering follows the article chapters
        # 2.1.1 Radar geometry
        theta_iRad = image.select('angle').multiply(np.pi / 180)  #地面入射角度转为弧度
        phi_iRad = ee.Image.constant(heading).multiply(np.pi / 180) #方位角转弧度

        # 2.1.2 Terrain geometry
        alpha_sRad = ee.Terrain.slope(elevation).select('slope').multiply(
            np.pi / 180).setDefaultProjection(proj).clip(geom)          # 坡度(与地面夹角)
        phi_sRad = ee.Terrain.aspect(elevation).select('aspect').multiply(
            np.pi / 180).setDefaultProjection(proj).clip(geom)          # 坡向角，(坡度陡峭度)坡与正北方向夹角(陡峭度)，从正北方向起算，顺时针计算角度

        # we get the height, for export
        height = elevation.setDefaultProjection(proj).clip(geom)

        # 2.1.3 Model geometry
        #reduce to 3 angle
        phi_rRad = phi_iRad.subtract(phi_sRad)     # (飞行方向角度-坡度陡峭度)飞行方向与坡向之间的夹角

        # 分解坡度，从一维分解为二维，
        alpha_rRad = (alpha_sRad.tan().multiply(phi_rRad.cos())).atan()     # 距离向分解
        alpha_azRad = (alpha_sRad.tan().multiply(phi_rRad.sin())).atan()    # 方位向分解

        # local incidence angle (eq. 4)
        theta_liaRad = (alpha_azRad.cos().multiply(
            (theta_iRad.subtract(alpha_rRad)).cos())).acos()               # LIA
        theta_liaDeg = theta_liaRad.multiply(180 / np.pi)                  # LIA转弧度

        # 2.2，根据入射角辐射校正
        # Gamma_nought
        gamma0 = sigma0Pow.divide(theta_iRad.cos())                        # 将原有反射信号通过入射角度进行削弱或增强
        gamma0dB = ee.Image.constant(10).multiply(gamma0.log10()).select(
            ['VV', 'VH'], ['VV_gamma0', 'VH_gamma0'])
        ratio_gamma = (gamma0dB.select('VV_gamma0').subtract(
            gamma0dB.select('VH_gamma0')).rename('ratio_gamma0'))

        if model == 'volume':
            scf = _volumetric_model_SCF(theta_iRad, alpha_rRad)

        if model == 'surface':
            scf = _surface_model_SCF(theta_iRad, alpha_rRad, alpha_azRad)

        # apply model for Gamm0_f
        gamma0_flat = gamma0.divide(scf)
        gamma0_flatDB = (ee.Image.constant(10).multiply(
            gamma0_flat.log10()).select(['VV', 'VH'],['VV_gamma0flat', 'VH_gamma0flat']))
        masks = _masking(alpha_rRad, theta_iRad, buffer)

        # calculate the ratio for RGB vis
        ratio_flat = (gamma0_flatDB.select('VV_gamma0flat').subtract(
            gamma0_flatDB.select('VH_gamma0flat')).rename('ratio_gamma0flat'))

        return (image.rename(['VV_sigma0', 'VH_sigma0','incAngle'])
                .addBands(gamma0dB)
                .addBands(ratio_gamma)
                .addBands(gamma0_flatDB)
                .addBands(ratio_flat)
                .addBands(alpha_rRad.rename('alpha_rRad'))
                .addBands(alpha_azRad.rename('alpha_azRad'))
                .addBands(phi_sRad.rename('aspect'))
                .addBands(alpha_sRad.rename('slope'))
                .addBands(theta_iRad.rename('theta_iRad'))
                .addBands(theta_liaRad.rename('theta_liaRad'))
                .addBands(masks).addBands(height.rename('elevation')))

    # run and return correction
    if type(collection) == ee.imagecollection.ImageCollection:
        return collection.map(_correct)
    elif type(collection) == ee.image.Image:
        return _correct(collection)
    else:
        print('Check input type, only image collection or image can be input')

# 几何畸变定位与校正

In [13]:
# 采用函数进行几何畸变矫正
# s1_col = slope_correction(s1_col, DEM, Model, buffer=Buffer)
# 分离升降轨
s1_descending = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')).first()
s1_ascending = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).first()

In [17]:
image = s1_descending
elevation = DEM

# get the image geometry and projection
geom = image.geometry()
proj = image.select(1).projection()

# 计算雷达方位向，ee.Terrain.aspect()本是用来计算坡度，该函数返回每个像素点相对于正北方向的坡度角度值
heading = (ee.Terrain.aspect(image.select('angle')).reduceRegion(
    ee.Reducer.mean(), geom, 1000).get('aspect'))

# Sigma0 to Power of input image，Sigma0是指雷达回波信号的强度,dB转信号强度
sigma0Pow = ee.Image.constant(10).pow(image.divide(10.0))

# the numbering follows the article chapters
# 2.1.1 Radar geometry
theta_iRad = image.select('angle').multiply(np.pi / 180)  #地面入射角度转为弧度
phi_iRad = ee.Image.constant(heading).multiply(np.pi / 180) #方位角转弧度

# 2.1.2 Terrain geometry
alpha_sRad = ee.Terrain.slope(elevation).select('slope').multiply(
    np.pi / 180).setDefaultProjection(proj).clip(geom)          # 坡度(与地面夹角)
phi_sRad = ee.Terrain.aspect(elevation).select('aspect').multiply(
    np.pi / 180).setDefaultProjection(proj).clip(geom)          # 坡向角，(坡度陡峭度)坡与正北方向夹角(陡峭度)，从正北方向起算，顺时针计算角度

# we get the height, for export
height = elevation.setDefaultProjection(proj).clip(geom)

# 2.1.3 Model geometry
#reduce to 3 angle
phi_rRad = phi_iRad.subtract(phi_sRad)     # (飞行方向角度-坡度陡峭度)飞行方向与坡向之间的夹角

# 分解坡度，从一维分解为二维，
alpha_rRad = (alpha_sRad.tan().multiply(phi_rRad.cos())).atan()     # 距离向分解
alpha_azRad = (alpha_sRad.tan().multiply(phi_rRad.sin())).atan()    # 方位向分解

# local incidence angle (eq. 4)
theta_liaRad = (alpha_azRad.cos().multiply(
    (theta_iRad.subtract(alpha_rRad)).cos())).acos()               # LIA
theta_liaDeg = theta_liaRad.multiply(180 / np.pi)                  # LIA转弧度

# ------------------------------RS几何畸变区域---------------------------------

# layover, where slope > radar viewing angle
layover = alpha_rRad.lt(theta_iRad).rename('layover')

# shadow
ninetyRad = ee.Image.constant(90).multiply(np.pi / 180)
shadow = alpha_rRad.gt(
    ee.Image.constant(-1).multiply(
        ninetyRad.subtract(theta_iRad))).rename('shadow')

def rgbmask(layover,shadow):
    r = ee.Image.constant(0).select([0], ['red'])
    g = ee.Image.constant(0).select([0], ['green'])  
    b = ee.Image.constant(0).select([0], ['blue'])
    
    r = r.where(layover, 255)
    g = g.where(shadow, 255)
    
    return ee.Image.cat([r, g, b]).byte()


# combine layover and shadow,因为shadow和layover都是0
no_data_mask = layover.And(shadow).rename('no_data_mask')
no_data_maskrgb = rgbmask(layover,shadow)

image = image.rename(['VV_sigma0', 'VH_sigma0','incAngle']).addBands(layover.addBands(shadow).addBands(no_data_mask).addBands(no_data_maskrgb))

## 普通展示用于测试

In [15]:
Map = geemap.Map()
Map.centerObject(AOI, zoom=12)
Map.addLayer(image.select('red','green','blue'), {'min':0,'max':255}, 'no_data_maskrgb')
Map.addLayer(image.select('shadow'), {'min':0,'max':1}, 'shadow')
Map.addLayer(image.select('no_data_mask'), {'min':0,'max':1}, 'no_data_mask')
Map

Map(center=[28.79865580293725, 96.6115336058235], controls=(WidgetControl(options=['position', 'transparent_bg…

## 展示,生成左右对照Basemap与畸变检测对照

In [60]:
Map = geemap.Map()
Map.centerObject(AOI, zoom=12)
left_layer = geemap.ee_tile_layer(image.select('VV_sigma0'), {'min':-40,'max':5}, 'Origin')
right_layer =  geemap.ee_tile_layer(image.select('no_data_mask'), {'min':0,'max':1}, 'Correct')
Map.split_map(left_layer,right_layer)
Map

Map(center=[29.391814573124467, 97.02212178683176], controls=(ZoomControl(options=['position', 'zoom_in_text',…

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

...

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

In [25]:
Map = geemap.Map()
Map.centerObject(AOI, zoom=12)
left_layer = geemap.ee_tile_layer(s1_descending, {'min':-40,'max':5}, 'NLCD 2001')
Map.split_map(left_layer,left_layer)
Map

Map(center=[29.391814573124467, 97.02212178683176], controls=(ZoomControl(options=['position', 'zoom_in_text',…

# 被动阴影和叠掩

## 沿着雷达入射角，构建近似于等高线，在登高线方向做局部最高点，且在坡两端做法线夹角

In [160]:
def create_contours(image, min_value, max_value, interval, kernel=None, region=None, values=None,gaussion=False):
    if not isinstance(image, ee.Image):
        raise TypeError("The image must be an ee.Image.")
    if region is not None:
        if isinstance(region, ee.FeatureCollection) or isinstance(region, ee.Geometry):
            pass
        else:
            raise TypeError(
                "The region must be an ee.Geometry or ee.FeatureCollection."
            )
    if kernel is None:
        kernel = ee.Kernel.gaussian(5, 3)
    if isinstance(values, list):
        values = ee.List(values)
    elif isinstance(values, ee.List):
        pass

    if values is None:
        values = ee.List.sequence(min_value, max_value, interval)
    def contouring(value):
        if gaussion:
            mycountour = (
               image.convolve(kernel)                # 高斯卷积
              .subtract(ee.Image.constant(value))         # 构建零交叉位置
              .zeroCrossing()                   # 生成零交叉位置掩码
              .multiply(ee.Image.constant(value).toFloat())    #
          )
        else:
               mycountour = (
                  image.subtract(ee.Image.constant(value))         # 构建零交叉位置
                  .zeroCrossing()                   # 生成零交叉位置掩码
                  .multiply(ee.Image.constant(value).toFloat())    #
              )
        return mycountour.mask(mycountour)
    contours = values.map(contouring)
    return contours

In [212]:
Map = geemap.Map()
Map.centerObject(AOI, zoom=8)

# 设置离散的颜色调色板列表（以角度为单位）
palette = ['blue', 'green', 'yellow', 'red']
MinMax_Angle = image.select('incAngle').reduceRegion(
                  reducer=ee.Reducer.minMax(),
                  geometry=s1_ascending.geometry(),
                  scale=10,
                  bestEffort=True
                                )
min =  MinMax_Angle.get('incAngle_min')
max = MinMax_Angle.get('incAngle_max')
interval = 0.1

discrete_vis = {'min':min, 'max':max ,'palette': palette}

contours = create_contours(image.select('incAngle'),min ,max, interval, region=None,gaussion=False)

contours_image = ee.ImageCollection(contours).mosaic()

Map.addLayer(contours_image, {'min': min, "max":max, 'color':'black'}, 'contours')
Map.addLayer(image.select('incAngle'), discrete_vis, 'incAngle')

Map # 显示地图窗口

Map(center=[29.391814573124467, 97.02212178683176], controls=(WidgetControl(options=['position', 'transparent_…

In [220]:
test = ee.Image(contours.get(100))
masked = test.mask().clip(test.geometry())

In [278]:
masked

In [277]:
roi = Map.draw_last_feature
roi = roi.geometry()
masked.reduceRegion(
          reducer=ee.Reducer.toList(),
          geometry=roi,   #这里范围大会出错
          scale=10,
          bestEffort=True)

In [239]:
# 将图像转化为点要素集,每个点代表一个像素:
pixel_points = test.sample(**{'region': test.geometry(), 'geometries': True})

In [245]:
image = test.addBands(ee.Image.pixelLonLat())
data = image.reduceRegion(
  reducer = ee.Reducer.toList(),
  geometry = test.geometry(),
  scale = 10,
  maxPixels = 1e13
)

In [273]:
roi = Map.draw_last_feature
masked = masked.reproject(roi.geometry()).clip(roi)

In [274]:
masked

In [262]:
masked.reduceToVectors(scale=10, geometryType='polygon')

In [175]:
# 显示nodata

# 删除nodata区域

In [263]:
Map.addLayer(masked, {'min': 0, "max":1, 'color':'red'}, 'test')

In [201]:
Map.addLayer(Vectors, {'color': 'red'}, 'Vectors')

EEException: Image.reduceToVectors: Need 1+1 bands for Reducer.max, image has 1.

In [187]:
def image2vector(result,resultband=0,GLarea=1.,FilterBound=None):

  # 分类图转为矢量并删除背景，添加select(0)会减少bug，不晓得为啥
    if GLarea>20:
        Vectors = result.select(0).reduceToVectors(scale=30, geometryType='polygon', eightConnected=True)
    else:
        Vectors = result.select(0).reduceToVectors(scale=10, geometryType='polygon', eightConnected=True)

    # 提取分类结果,并合并为一个矢量
    Extract = NoBackground_Vectors.filterBounds(FilterBound)
    Union_ex = ee.Feature(Extract.union(1).first())

    return Union_ex

## 去除异常曲线，其余曲线求平均斜率，与法线角

In [13]:
def Geemap_export(filename,collection=False,image=False,region=None,scale=10):
    if collection:
        # 这里导出时候使用region设置AOI，否则可能因为坐标系问题(未确定)，出现黑边问题
        geemap.ee_export_image_collection(collection,
                          out_dir=os.path.dirname(out_dir),
                          format = "ZIPPED_GEO_TIFF",region=region,scale=scale)
        print('collection save right')
    elif image:
        geemap.ee_export_image(image,
                  filename=filename,
                  scale=scale, region=region, file_per_band=False,timeout=1500)
        print('image save right')
    else:
        print('Erro:collection && image must have one False')

def cut_geometry(geometry,block_size:float=0.05):
    '''
    block_size 定义方块大小(地理坐标系度)
    '''
    # 计算边界
    bounds = ee.List(ee.List(geometry.bounds().coordinates()).get(0))

    # 计算geometry的宽度和高度
    width = ee.Number(ee.List(bounds.get(2)).get(0)).subtract(ee.Number(ee.List(bounds.get(0)).get(0)))
    height = ee.Number(ee.List(bounds.get(2)).get(1)).subtract(ee.Number(ee.List(bounds.get(0)).get(1)))

    # 计算行和列的数量
    num_rows = height.divide(block_size).ceil()
    num_cols = width.divide(block_size).ceil()

    # 定义一个函数，用于生成512x512的方块
    def create_blocks(row, col):
        x_min = ee.Number(ee.List(bounds.get(0)).get(0)).add(col.multiply(block_size))
        y_min = ee.Number(ee.List(bounds.get(0)).get(1)).add(row.multiply(block_size))
        x_max = x_min.add(block_size)
        y_max = y_min.add(block_size)
        return ee.Geometry.Rectangle([x_min, y_min, x_max, y_max])

    # 生成方块列表
    block_list = []
    for row in range(num_rows.getInfo()):
        for col in range(num_cols.getInfo()):
            block = create_blocks(ee.Number(row), ee.Number(col))
            block_list.append(block)
    return block_list

def contouring(value):
    mycountour = (
        image#.convolve(kernel)                # 高斯卷积
        .subtract(ee.Image.constant(value))         # 构建零交叉位置
        .zeroCrossing()                   # 生成零交叉位置掩码
        .multiply(ee.Image.constant(value).toFloat())    #
    )
    return mycountour.mask(mycountour)

# 卷积核
kernel = ee.Kernel.gaussian(5, 3)
min_value = min
max_value = max
interval = 0.1
values = ee.List.sequence(min_value, max_value, interval)
image = s1_ascending.select('incAngle')
contours = values.map(contouring)

In [14]:
value = values.get(100)
test = contouring(value)